In [1]:
import torch
import torch.nn
import torch.nn.functional as F
import tqdm
import logging
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence
import torch.nn as nn
import torch.optim as optim
from torch.utils.tensorboard import SummaryWriter
import torch
import unicodedata
import string
from tqdm import tqdm
from pathlib import Path
from typing import List
from itertools import chain
import time
import re
from torch.utils.tensorboard import SummaryWriter
import torch
import torch.nn as nn
from torch.nn.modules import dropout
from  torch.nn.utils.rnn import pack_padded_sequence
from  torch.nn.utils.rnn import pad_packed_sequence
import torch.nn.functional as F
import sentencepiece as sp
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#all_string=string.ascii_letters+'čḍɛǧɣḥṛṣṭẓ'+'čḍɛǧɣḥṛṣṭẓ'.upper()+'ⴰⴱⵛⴷⴻⴼⴳⵀⵉⵊⴽⵍⵎⵏⵇⵔⵙⵜⵓⵡⵅⵢⵣ'

In [3]:
def normalize(s):
    return re.sub(' +',' ', "".join(c if c in string.ascii_letters else " "
         for c in unicodedata.normalize('NFD', s.lower().strip())
         if  c in string.ascii_letters+" "+string.punctuation)).strip()

In [4]:
#########Load File#####################
FILE = "./data/en-fra.txt"
############################################################################################
with open(FILE) as f:
    lines = f.readlines()
original=[]
destination=[]
for line in tqdm(lines):
    for s in line.split("\n"):
        if len(s)<1:continue
        orig,dest=map(normalize,s.split("\t")[:2])
        original.append(orig)
        destination.append(dest)

with open('./data/en.txt', 'w') as f:
        for ori in original:
            f.write(ori + "\n")

with open('./data/fr.txt', 'w') as f:
        for dest in destination:
            f.write(dest + "\n")

100%|██████████| 170651/170651 [00:03<00:00, 43027.85it/s]


In [5]:
#create our English SentencePiece model
sp.SentencePieceTrainer.Train(input="./data/en.txt",model_prefix='./data/en',vocab_size=5000)
en = spm.SentencePieceProcessor(model_file='./data/en.model')

NameError: name 'spm' is not defined

In [ ]:
sp.SentencePieceTrainer.Train(input="./data/fr.txt",model_prefix='./data/fr',vocab_size=5000)
fr = spm.SentencePieceProcessor(model_file='./data/fr.model')

In [6]:

class TradDataset():
    def __init__(self,dataOriginal,dataDestination,vocOrig,vocDest,adding=True,max_len=10):
        self.sentences =[]
        for orig,dist in zip(dataOriginal,dataDestination):
            if len(orig)>max_len: continue
            self.sentences.append((torch.tensor(vocOrig.encode(orig, out_type=int)),torch.tensor(vocDest.encode(dist, out_type=int))))
    def __len__(self):return len(self.sentences)
    def __getitem__(self,i): return self.sentences[i]



def collate(batch):
    orig,dest = zip(*batch)
    o_len = torch.tensor([len(o) for o in orig])
    d_len = torch.tensor([len(d) for d in dest])
    return pad_sequence(orig),o_len,pad_sequence(dest),d_len

In [7]:
class Encoder(nn.Module):
    #encodeur: un embedding du vocabulaire d'origine puis un gru
    def __init__(self, vocab_origine,emb_size, hidden_size):
        super(Encoder,self).__init__()
        self.vocab_origine=vocab_origine
        self.hidden_size=hidden_size
        self.emb_size=emb_size
        self.embedding = nn.Embedding(vocab_origine, emb_size,padding_idx=PAD)
        self.gru = nn.GRU(emb_size, hidden_size)

    def forward(self, input,lengths,hidden=None):
        embedded = self.embedding(input) #Input pad_sequence
        embedded = pack_padded_sequence(embedded,lengths.cpu(),enforce_sorted=False)                                                                                                       
        outputs, hidden = self.gru(embedded,hidden)
        output,_  = torch.nn.utils.rnn.pad_packed_sequence(outputs) 
        return output, hidden

In [8]:
class Decoder(nn.Module):

    #décodeur : un embedding du vocabulaire de destination, puis un GRU 
    # suivi d'un réseau linéaire pour le décodage de l'état latent (et un softmax pour terminer)

    def __init__(self,vocab_destination,emb_size,hidden_size,bidirectional=False):
        super(Decoder,self).__init__()
        self.vocab_destination=vocab_destination
        self.hidden_size = hidden_size
        self.emb_size=emb_size
        self.embedding = nn.Embedding(vocab_destination, emb_size,padding_idx=PAD)
        self.gru = nn.GRU(emb_size, hidden_size)
        self.decoder = nn.Linear(hidden_size  , vocab_destination)
    
    def forward(self,input,lengths,hidden):
        #For teacher forcing
        """
        input: Length x batch 
        hidden: tensor of shape (1, batch, hidden_size)
        """
        target = torch.tensor([[SOS]*input.shape[1]],device=device) # 1 x Batch
        target=self.embedding(target)#1 x Batch x emb_size 
        _, decoder_hidden = self.gru(target, hidden)  #1 x batch x hidden_size , 1 x batch x hidden_size
        target=self.embedding(input)# Length x batch x emb_size
        target = pack_padded_sequence(target,lengths.cpu(),enforce_sorted=False)        
        outputs,hidden=self.gru(target,decoder_hidden)
        output,_  = torch.nn.utils.rnn.pad_packed_sequence(outputs) 
        return F.log_softmax(self.decoder(output),dim=-1)    #(Length x Batch)

    def generate(self,hidden,lenseq=None):
        #For constraint mode
        """
        lenseq:length
        hidden:1 x Batch x hidden_size
        """
        batch_size=hidden.shape[1]
        decoder_outputs = torch.full(size=[lenseq, batch_size, self.vocab_destination], fill_value=PAD,dtype=torch.float,device=device)
        
        for i in range(batch_size):
            target = torch.tensor([[SOS]],device=device)
            hidden_example=hidden[:,i,:].unsqueeze(1) #1 x 1 x hidden_size
            
            for j in range(lenseq):
                target = self.embedding(target).view(1, 1, -1) # one example and one word -> 1 x 1 x emb_size
                output, hidden_example = self.gru(target, hidden_example)
                decoder_output = F.log_softmax(self.decoder(hidden_example), dim=-1).squeeze()# vocab_size
                indice_max = decoder_output.argmax() #1
                target = indice_max.detach() #1
                decoder_outputs[j,i]=decoder_output
                
                if target.item() == EOS:
                    break
        return decoder_outputs

In [17]:
logging.basicConfig(level=logging.INFO)
FILE_ORIGINAL = "./data/en.txt"
FILE_DESTINATION = "./data/fr.txt"
############################################################################################
with open(FILE_ORIGINAL) as f:
    dataOriginal = f.readlines()
    
with open(FILE_DESTINATION) as f:
    dataDestination = f.readlines()
    
############################################################################################ 
#create our English SentencePiece model
sp.SentencePieceTrainer.Train(input="./data/en.txt",model_prefix='./data/en',vocab_size=5000)
en = sp.SentencePieceProcessor(model_file='./data/en.model')

#create our French SentencePiece model
sp.SentencePieceTrainer.Train(input="./data/fr.txt",model_prefix='./data/fr',vocab_size=5000)
fr = sp.SentencePieceProcessor(model_file='./data/fr.model')
############################################################################################
dataOriginal = [dataOriginal[x] for x in torch.randperm(len(dataOriginal))]
dataDestination = [dataDestination[x] for x in torch.randperm(len(dataDestination))]
idxTrain = int(0.8*len(lines))
############################################################################################
MAX_LEN=10
BATCH_SIZE=64
############################################################################################
datatrain = TradDataset(dataOriginal[:idxTrain],dataDestination[:idxTrain],en,fr,max_len=MAX_LEN)
datatest = TradDataset(dataOriginal[idxTrain:],dataDestination[idxTrain:],en,fr,max_len=MAX_LEN)
train_loader = DataLoader(datatrain, collate_fn=collate, batch_size=BATCH_SIZE, shuffle=True)
test_loader = DataLoader(datatest, collate_fn=collate, batch_size=BATCH_SIZE, shuffle=True)
############################################################################################
VOCAB_ORIGINE=en.vocab_size()
VOCAB_DESTINATION=fr.vocab_size()
HIDDEN_SIZE=2048
EMB_SIZE=256
NB_EPOCH=100

PAD = en.pad_id()
EOS = en.eos_id()
SOS = en.bos_id()

In [18]:
############################################################################################
encoder=Encoder(VOCAB_ORIGINE,EMB_SIZE, HIDDEN_SIZE).to(device)
decoder=Decoder(VOCAB_DESTINATION,EMB_SIZE, HIDDEN_SIZE).to(device)
############################################################################################
loss_fn=nn.NLLLoss(ignore_index=PAD)
############################################################################################

In [19]:
optimizer=optim.RMSprop(chain(encoder.parameters(),decoder.parameters()), lr=0.001)
#scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.9)

In [ ]:
teacher_forcing_ratio = 0.5

for i in range(NB_EPOCH):
    acc=0
    for orig , o_len, dest ,d_len in train_loader:
        enc_output, enc_hidden = encoder(orig.to(device), o_len.to(device))
        use_teacher_forcing = True if torch.rand(1) < teacher_forcing_ratio else False
        
        if use_teacher_forcing:
            dec_output=decoder(dest.to(device),d_len.to(device),enc_hidden) #OK 
        else:
            dec_output=decoder.generate(enc_hidden,lenseq=max(d_len))
        loss = loss_fn(dec_output.permute(0,2,1), dest.to(device))
        with torch.no_grad():
            acc+=loss
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    print(f'Loss/Train {acc/len(train_loader)} a l"epoche {i}')
    #scheduler.step()

In [ ]:
encoder.load_state_dict(torch.load('./encoder_saved.pt'))
decoder.load_state_dict(torch.load('./decoder_saved.pt'))

In [ ]:
orig , o_len, dest ,d_len = next(iter(train_loader))
enc_output, enc_hidden = encoder(orig, o_len)
dec_output=decoder.generate(enc_hidden,max(d_len)).argmax(dim=-1)    

In [ ]:
print(" ".join(vocEng.getwords(orig[1])))

In [ ]:
[vocBer.getwords(seq) for seq in dec_output.permute(1,0)][1]

In [ ]:
" " .join(vocBer.getwords(dest[1]))